In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
#from api_keys import p_key

# Incorporated citipy to determine city based on latitude and longitude
#from citipy import citipy

# Output File (CSV)
#output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
#lat_range = (-90, 90)
#lng_range = (-180, 180)

### API Calls

In [12]:
#Ryan API

In [13]:
#Nicole API

In [14]:
#Kayla API
import requests

url = "https://hotels-com-free.p.rapidapi.com/srle/listing/v1/brands/hotels.com"

querystring = {"lat":"37.788719679657554","lon":"-122.40057774847898","checkIn":"2021-03-15","checkOut":"2021-03-19","rooms":"2","locale":"en_US","currency":"USD","pageNumber":"1"}

headers = {
    'x-rapidapi-key': "7d7c570015msh06f88cde872d5e5p15c904jsn38364700fb06",
    'x-rapidapi-host': "hotels-com-free.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"result":"OK","data":{"body":{"header":"22 Anthony St, San Francisco, CA 94105, USA","query":{"destination":{"id":"1493604","value":"22 Anthony St, San Francisco, CA 94105, USA","resolvedLocation":"GEO_LOCATION:22 Anthony St, San Francisco, CA 94105, USA|37.78880310058594|-122.40035247802734:GEOCODE:UNKNOWN"}},"searchResults":{"totalCount":411,"results":[{"id":693927,"name":"Hotel Ikon","starRating":2.5,"urls":{},"address":{"streetAddress":"325 Sutter Street","extendedAddress":"","locality":"San Francisco","postalCode":"94108","region":"CA","countryName":"United States","countryCode":"us","obfuscate":false},"welcomeRewards":{"collect":true},"guestReviews":{"unformattedRating":3.3,"rating":"3.3","total":187,"scale":5,"badge":"good","badgeText":"Good"},"landmarks":[{"label":"22 Anthony St, San Francisco, CA 94105, USA","distance":"0.3 miles"},{"label":"San Francisco","distance":"0.2 miles"}],"geoBullets":[],"ratePlan":{"price":{"current":"$65","exactCurrent":65.2,"old":"$95","info":"nig